In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM , Dropout ,Flatten, Input, Activation
from keras.utils import np_utils


Using TensorFlow backend.


In [20]:
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [21]:
emo_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

for ix in emo_dict.keys():
    print(ix),
    print(emoji.emojize(emo_dict[ix], use_aliases=True))


0 ❤
1 ⚾
2 😄
3 😞
4 🍴


In [22]:
x_train = train[0]
y_train= train[1]

x_test = test[0]
y_test = test[1]

print x_train.shape, y_train.shape,x_test.shape, y_test.shape
print ("-------------------------")
print x_train[0],y_train[0]

(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [23]:
for ix in range(x_train.shape[0]):
    x_train[ix] = x_train[ix].split()
    
for ix in range(x_test.shape[0]):
    x_test[ix] = x_test[ix].split()

/home/arif/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/arif/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [24]:
y_train=np_utils.to_categorical(y_train)


In [25]:
## Finding max length pf sentence array

In [28]:
np.unique([len(ix) for ix in x_train],return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4]))

In [29]:
np.unique([len(ix) for ix in x_test],return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]), array([ 3, 12, 16, 17,  3,  4,  1]))

In [31]:
embeddings_ind = {}

f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coeff = values[1:]
    embeddings_ind[word] = coeff
f.close()

In [33]:
embeddings_mat_train = np.zeros((x_train.shape[0],10,50))
embeddings_mat_test= np.zeros((x_test.shape[0],10,50))

In [45]:
for ix in range(x_train.shape[0]):
    for iy in range(len(x_train[ix])):
        embeddings_mat_train[ix][iy] = embeddings_ind[x_train[ix][iy].lower()]

In [46]:
for ix in range(x_test.shape[0]):
    for iy in range(len(x_test[ix])):
        embeddings_mat_test[ix][iy] = embeddings_ind[x_test[ix][iy].lower()]

In [61]:
model= Sequential()
model.add(SimpleRNN(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.25))
model.add(SimpleRNN(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_8 (SimpleRNN)     (None, 10, 64)            7360      
_________________________________________________________________
dropout_7 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_9 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_4 (Activation)    (None, 5)                 0         
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
_________________________________________________________________


In [62]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [63]:
model.fit(embeddings_mat_train,y_train,epochs=50,batch_size=32,shuffle=True)

Epoch 1/50
132/132 [==============================] - 1s 5ms/step - loss: 1.7955 - acc: 0.2273
Epoch 2/50
132/132 [==============================] - 0s 284us/step - loss: 1.6451 - acc: 0.2955
Epoch 3/50
132/132 [==============================] - 0s 351us/step - loss: 1.5907 - acc: 0.3788
Epoch 4/50
132/132 [==============================] - 0s 314us/step - loss: 1.4598 - acc: 0.3712
Epoch 5/50
132/132 [==============================] - 0s 305us/step - loss: 1.3303 - acc: 0.4394
Epoch 6/50
132/132 [==============================] - 0s 327us/step - loss: 1.3667 - acc: 0.4545
Epoch 7/50
132/132 [==============================] - 0s 284us/step - loss: 1.1775 - acc: 0.5758
Epoch 8/50
132/132 [==============================] - 0s 293us/step - loss: 1.1445 - acc: 0.5530
Epoch 9/50
132/132 [==============================] - 0s 300us/step - loss: 1.1076 - acc: 0.5909
Epoch 10/50
132/132 [==============================] - 0s 283us/step - loss: 1.0614 - acc: 0.6061
Epoch 11/50
132/132 [==========

In [71]:
pred = model.predict_classes(embeddings_mat_test)
print(float(sum(y_test==preds))/embeddings_mat_test.shape[0])=

0.660714285714


In [65]:
for ix in range(embeddings_mat_test.shape[0]):
    
    if pred[ix] != y_test[ix]:
        print(ix)
        print test[0][ix],
        print emoji.emojize(emo_dict[pred[ix]], use_aliases=True),
        print emoji.emojize(emo_dict[y_test[ix]], use_aliases=True)

2
['he', 'got', 'a', 'raise'] 🍴 😄
5
['he', 'is', 'a', 'good', 'friend'] 😞 ❤
6
['I', 'am', 'upset'] 😞 ❤
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 🍴 ❤
11
['work', 'is', 'hard'] ❤ 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] 😄 😞
13
['are', 'you', 'serious', 'ha', 'ha'] 😞 😄
17
['Congratulation', 'for', 'having', 'a', 'baby'] 😞 😄
18
['stop', 'messing', 'around'] ❤ 😞
21
['you', 'brighten', 'my', 'day'] 🍴 😄
23
['she', 'is', 'a', 'bully'] ❤ 😞
26
['I', 'worked', 'during', 'my', 'birthday'] 🍴 😞
27
['My', 'grandmother', 'is', 'the', 'love', 'of', 'my', 'life'] 😞 ❤
28
['enjoy', 'your', 'break'] 🍴 😄
29
['valentine', 'day', 'is', 'near'] 😄 ❤
30
['I', 'miss', 'you', 'so', 'much'] 😞 ❤
34
['will', 'you', 'be', 'my', 'valentine'] 🍴 ❤
35
['he', 'can', 'pitch', 'really', 'well'] 😄 ⚾
41
['I', 'like', 'your', 'jacket'] ❤ 😄
45
['I', 'love', 'to', 'the', 'stars', 'and', 'back'] ⚾ ❤
50
['yesterday', 'we', 'lost', 'again'] 🍴 😞
51
['family', 'is', 'all', 'I', 'have'] 😞 ❤
55
['I', 'did',

## Using LSTM

In [66]:
model_new=Sequential()
model_new.add(LSTM(128,input_shape=(10,50),return_sequences=True))
model_new.add(Dropout(0.5))

model_new.add(LSTM(128,return_sequences=False))
model_new.add(Dropout(0.5))
model_new.add(Dense(5))
model_new.add(Activation('softmax'))
model_new.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 128)           91648     
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
Total params: 223,877
Trainable params: 223,877
Non-trainable params: 0
_________________________________________________________________


In [67]:
model_new.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [68]:
model_new.fit(embeddings_mat_train,y_train,epochs=50,batch_size=32,shuffle=True)

Epoch 1/50
132/132 [==============================] - 2s 13ms/step - loss: 1.5930 - acc: 0.2803
Epoch 2/50
132/132 [==============================] - 0s 987us/step - loss: 1.5119 - acc: 0.2879
Epoch 3/50
132/132 [==============================] - 0s 1ms/step - loss: 1.4765 - acc: 0.3106
Epoch 4/50
132/132 [==============================] - 0s 1ms/step - loss: 1.4066 - acc: 0.4470
Epoch 5/50
132/132 [==============================] - 0s 1ms/step - loss: 1.2945 - acc: 0.4773
Epoch 6/50
132/132 [==============================] - 0s 1ms/step - loss: 1.1900 - acc: 0.5833
Epoch 7/50
132/132 [==============================] - 0s 1ms/step - loss: 1.0440 - acc: 0.6212
Epoch 8/50
132/132 [==============================] - 0s 1ms/step - loss: 0.9404 - acc: 0.6212
Epoch 9/50
132/132 [==============================] - 0s 1ms/step - loss: 0.8041 - acc: 0.6288
Epoch 10/50
132/132 [==============================] - 0s 1ms/step - loss: 0.8048 - acc: 0.6970
Epoch 11/50
132/132 [=========================

In [69]:
preds = model_new.predict_classes(embeddings_mat_test)
print(float(sum(y_test==preds))/embeddings_mat_test.shape[0])

0.660714285714


In [70]:
for ix in range(embeddings_mat_test.shape[0]):
    
    if preds[ix] != y_test[ix]:
        print(ix)
        print test[0][ix],
        print emoji.emojize(emo_dict[preds[ix]], use_aliases=True),
        print emoji.emojize(emo_dict[y_test[ix]], use_aliases=True)

2
['he', 'got', 'a', 'raise'] 😞 😄
3
['she', 'got', 'me', 'a', 'present'] 😞 ❤
5
['he', 'is', 'a', 'good', 'friend'] 😄 ❤
6
['I', 'am', 'upset'] 😞 ❤
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 😄 ❤
11
['work', 'is', 'hard'] 😄 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] ❤ 😞
13
['are', 'you', 'serious', 'ha', 'ha'] 😞 😄
16
['work', 'is', 'horrible'] 😄 😞
19
['any', 'suggestions', 'for', 'dinner'] 😄 🍴
21
['you', 'brighten', 'my', 'day'] ❤ 😄
23
['she', 'is', 'a', 'bully'] ❤ 😞
26
['I', 'worked', 'during', 'my', 'birthday'] 😄 😞
28
['enjoy', 'your', 'break'] 😞 😄
29
['valentine', 'day', 'is', 'near'] 😄 ❤
40
['I', 'will', 'go', 'dance'] ⚾ 😄
41
['I', 'like', 'your', 'jacket'] ❤ 😄
48
['I', 'want', 'to', 'joke'] ❤ 😄
55
['I', 'did', 'not', 'have', 'breakfast'] ❤ 😞
